# Improved Churn Prediction Model

This notebook implements several improvements to the churn prediction model:
1. Advanced data preprocessing
2. Feature engineering
3. Multiple model comparison
4. Hyperparameter tuning
5. Handling class imbalance
6. Comprehensive model evaluation

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, precision_recall_curve, average_precision_score
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
import xgboost as xgb
import joblib
import warnings
warnings.filterwarnings('ignore')

# Set style for plots
sns.set(style='whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)

## 1. Load and Explore the Dataset

In [ ]:
# Load the dataset
df = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')

# Display basic information
print(f"Dataset shape: {df.shape}")
print("\nFirst 5 rows:")
display(df.head())

# Check for missing values
print("\nMissing values:")
print(df.isnull().sum())

# Check data types
print("\nData types:")
print(df.dtypes)

In [ ]:
# Check for TotalCharges issues
print(f"TotalCharges unique values count: {df['TotalCharges'].nunique()}")
print(f"TotalCharges sample: {df['TotalCharges'].sample(5).values}")

# Convert TotalCharges to numeric
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
print(f"\nMissing values after conversion: {df['TotalCharges'].isnull().sum()}")

# Check class distribution
print("\nChurn distribution:")
churn_counts = df['Churn'].value_counts(normalize=True) * 100
print(churn_counts)

# Visualize churn distribution
plt.figure(figsize=(8, 6))
sns.countplot(x='Churn', data=df)
plt.title('Churn Distribution')
plt.ylabel('Count')
for i, count in enumerate(df['Churn'].value_counts()):
    plt.text(i, count + 100, f"{count} ({count/len(df):.1%})", ha='center')
plt.show()

## 2. Data Preprocessing and Feature Engineering

In [ ]:
# Drop customerID as it's not relevant for prediction
df = df.drop('customerID', axis=1)

# Fill missing values in TotalCharges with the median
df['TotalCharges'] = df['TotalCharges'].fillna(df['TotalCharges'].median())

# Feature Engineering

# 1. Create tenure-related features
df['tenure_group'] = pd.qcut(df['tenure'], 4, labels=['0-1 year', '1-3 years', '3-5 years', '5+ years'])

# 2. Create average monthly charge
df['avg_monthly_charges'] = df['TotalCharges'] / (df['tenure'] + 0.1)  # Adding 0.1 to avoid division by zero

# 3. Create charge difference (how much more/less than average)
df['charge_diff'] = df['MonthlyCharges'] - df['avg_monthly_charges']

# 4. Create service count feature
service_columns = ['PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 
                   'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies']

# Function to count services
def count_services(row):
    count = 0
    for col in service_columns:
        if row[col] not in ['No', 'No phone service', 'No internet service', 'DSL']:
            count += 1
    return count

df['service_count'] = df.apply(count_services, axis=1)

# 5. Create binary flags for premium services
df['has_premium_tech'] = df['TechSupport'].apply(lambda x: 1 if x == 'Yes' else 0)
df['has_premium_security'] = df['OnlineSecurity'].apply(lambda x: 1 if x == 'Yes' else 0)
df['has_premium_support'] = df[['OnlineBackup', 'DeviceProtection']].apply(
    lambda x: 1 if 'Yes' in x.values else 0, axis=1
)
df['has_streaming'] = df[['StreamingTV', 'StreamingMovies']].apply(
    lambda x: 1 if 'Yes' in x.values else 0, axis=1
)

# 6. Create interaction features
df['tenure_x_monthly'] = df['tenure'] * df['MonthlyCharges']

# Display the new features
print("Dataset with new features:")
display(df.head())

# Visualize some of the new features
plt.figure(figsize=(12, 8))
sns.boxplot(x='tenure_group', y='MonthlyCharges', hue='Churn', data=df)
plt.title('Monthly Charges by Tenure Group and Churn Status')
plt.show()

plt.figure(figsize=(12, 8))
sns.boxplot(x='service_count', y='MonthlyCharges', hue='Churn', data=df)
plt.title('Monthly Charges by Service Count and Churn Status')
plt.show()

## 3. Feature Selection and Importance Analysis

In [ ]:
# Prepare data for feature importance analysis
# Convert categorical variables to numeric
df_encoded = df.copy()

# Binary encoding for Yes/No columns
binary_cols = ['Partner', 'Dependents', 'PhoneService', 'PaperlessBilling', 'Churn']
for col in binary_cols:
    df_encoded[col] = df_encoded[col].map({'Yes': 1, 'No': 0})

# One-hot encoding for categorical columns
cat_cols = ['gender', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup',
            'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies',
            'Contract', 'PaymentMethod', 'tenure_group']

df_encoded = pd.get_dummies(df_encoded, columns=cat_cols, drop_first=True)

# Prepare features and target
X = df_encoded.drop('Churn', axis=1)
y = df_encoded['Churn']

# Train a Random Forest for feature importance
rf_feat = RandomForestClassifier(n_estimators=100, random_state=42)
rf_feat.fit(X, y)

# Get feature importances
feature_importances = pd.DataFrame({
    'feature': X.columns,
    'importance': rf_feat.feature_importances_
}).sort_values('importance', ascending=False)

# Display top 20 features
print("Top 20 most important features:")
display(feature_importances.head(20))

# Visualize feature importances
plt.figure(figsize=(12, 10))
sns.barplot(x='importance', y='feature', data=feature_importances.head(20))
plt.title('Top 20 Feature Importances')
plt.tight_layout()
plt.show()

# Select top features (you can adjust the threshold)
top_features = feature_importances['feature'].head(25).tolist()
X_selected = X[top_features]

print(f"Selected {len(top_features)} top features for modeling")

## 4. Model Training and Evaluation

In [ ]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42, stratify=y)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Apply SMOTE to handle class imbalance
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train_scaled, y_train)

print(f"Original training set shape: {np.bincount(y_train)}")
print(f"Resampled training set shape: {np.bincount(y_train_smote)}")

# Define models to compare
models = {
    'Random Forest': RandomForestClassifier(random_state=42, class_weight='balanced'),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42),
    'XGBoost': xgb.XGBClassifier(random_state=42, scale_pos_weight=len(y_train[y_train==0])/len(y_train[y_train==1])),
    'Logistic Regression': LogisticRegression(random_state=42, class_weight='balanced', max_iter=1000),
    'SVM': SVC(random_state=42, class_weight='balanced', probability=True)
}

# Train and evaluate each model
results = {}
for name, model in models.items():
    print(f"\nTraining {name}...")
    model.fit(X_train_smote, y_train_smote)
    
    # Predict on test set
    y_pred = model.predict(X_test_scaled)
    
    # Calculate metrics
    report = classification_report(y_test, y_pred, output_dict=True)
    results[name] = {
        'model': model,
        'accuracy': report['accuracy'],
        'precision_churn': report['1']['precision'],
        'recall_churn': report['1']['recall'],
        'f1_churn': report['1']['f1-score'],
        'report': classification_report(y_test, y_pred)
    }
    
    print(f"{name} Results:")
    print(f"Accuracy: {report['accuracy']:.4f}")
    print(f"Precision (Churn): {report['1']['precision']:.4f}")
    print(f"Recall (Churn): {report['1']['recall']:.4f}")
    print(f"F1 Score (Churn): {report['1']['f1-score']:.4f}")
    print("\nClassification Report:")
    print(results[name]['report'])
    
    # Plot confusion matrix
    plt.figure(figsize=(8, 6))
    cm = confusion_matrix(y_test, y_pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False,
                xticklabels=['Not Churn', 'Churn'],
                yticklabels=['Not Churn', 'Churn'])
    plt.title(f'Confusion Matrix - {name}')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.show()
    
    # Plot ROC curve
    if hasattr(model, 'predict_proba'):
        y_prob = model.predict_proba(X_test_scaled)[:, 1]
        fpr, tpr, _ = roc_curve(y_test, y_prob)
        roc_auc = auc(fpr, tpr)
        
        plt.figure(figsize=(8, 6))
        plt.plot(fpr, tpr, lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
        plt.plot([0, 1], [0, 1], 'k--', lw=2)
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title(f'ROC Curve - {name}')
        plt.legend(loc="lower right")
        plt.show()
        
        # Plot Precision-Recall curve
        precision, recall, _ = precision_recall_curve(y_test, y_prob)
        pr_auc = average_precision_score(y_test, y_prob)
        
        plt.figure(figsize=(8, 6))
        plt.plot(recall, precision, lw=2, label=f'PR curve (area = {pr_auc:.2f})')
        plt.xlabel('Recall')
        plt.ylabel('Precision')
        plt.title(f'Precision-Recall Curve - {name}')
        plt.legend(loc="lower left")
        plt.show()

## 5. Hyperparameter Tuning for Best Model

In [ ]:
# Find the best performing model based on F1 score for churn class
best_model_name = max(results, key=lambda x: results[x]['f1_churn'])
print(f"Best model based on F1 score for churn class: {best_model_name}")

# Define hyperparameter grid based on best model
if best_model_name == 'Random Forest':
    param_grid = {
        'n_estimators': [100, 200, 300],
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'class_weight': ['balanced', 'balanced_subsample']
    }
    model = RandomForestClassifier(random_state=42)
elif best_model_name == 'Gradient Boosting':
    param_grid = {
        'n_estimators': [100, 200, 300],
        'learning_rate': [0.01, 0.05, 0.1],
        'max_depth': [3, 5, 7],
        'min_samples_split': [2, 5, 10],
        'subsample': [0.8, 0.9, 1.0]
    }
    model = GradientBoostingClassifier(random_state=42)
elif best_model_name == 'XGBoost':
    param_grid = {
        'n_estimators': [100, 200, 300],
        'learning_rate': [0.01, 0.05, 0.1],
        'max_depth': [3, 5, 7],
        'min_child_weight': [1, 3, 5],
        'gamma': [0, 0.1, 0.2],
        'subsample': [0.8, 0.9, 1.0],
        'colsample_bytree': [0.8, 0.9, 1.0],
        'scale_pos_weight': [1, 3, 5]
    }
    model = xgb.XGBClassifier(random_state=42)
elif best_model_name == 'Logistic Regression':
    param_grid = {
        'C': [0.001, 0.01, 0.1, 1, 10, 100],
        'penalty': ['l1', 'l2', 'elasticnet', None],
        'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
        'class_weight': ['balanced', None]
    }
    model = LogisticRegression(random_state=42, max_iter=1000)
else:  # SVM
    param_grid = {
        'C': [0.1, 1, 10, 100],
        'gamma': ['scale', 'auto', 0.1, 0.01],
        'kernel': ['rbf', 'poly', 'sigmoid'],
        'class_weight': ['balanced', None]
    }
    model = SVC(random_state=42, probability=True)

# Perform grid search with cross-validation
print("\nPerforming hyperparameter tuning with GridSearchCV...")
grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    cv=StratifiedKFold(n_splits=5),
    scoring='f1',
    n_jobs=-1,
    verbose=1
)

# Fit grid search to the data
grid_search.fit(X_train_smote, y_train_smote)

# Get best parameters and model
print(f"\nBest parameters: {grid_search.best_params_}")
print(f"Best cross-validation score: {grid_search.best_score_:.4f}")

# Evaluate the best model
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test_scaled)

print("\nBest Model Evaluation:")
print(classification_report(y_test, y_pred))

# Plot confusion matrix for best model
plt.figure(figsize=(8, 6))
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False,
            xticklabels=['Not Churn', 'Churn'],
            yticklabels=['Not Churn', 'Churn'])
plt.title('Confusion Matrix - Best Model')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

## 6. Save the Best Model and Preprocessing Pipeline

In [ ]:
# Create a full preprocessing pipeline
# This will include all the preprocessing steps we did manually

# Save the column names for later use
feature_columns = X_selected.columns.tolist()

# Save the best model
joblib.dump(best_model, 'improved_churn_model.pkl')
joblib.dump(feature_columns, 'improved_churn_model_columns.pkl')
joblib.dump(scaler, 'improved_churn_model_scaler.pkl')

# Save the classification report
with open('improved_classification_report.txt', 'w') as f:
    f.write(classification_report(y_test, y_pred))

print("Model, feature columns, and scaler saved successfully!")

# Create a function to preprocess new data
def preprocess_for_prediction(df_input):
    """
    Preprocess a new customer dataframe for prediction.
    
    Args:
        df_input: DataFrame with customer data
        
    Returns:
        Preprocessed features ready for model prediction
    """
    df = df_input.copy()
    
    # Drop customerID if present
    if 'customerID' in df.columns:
        df = df.drop('customerID', axis=1)
    
    # Convert TotalCharges to numeric
    df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
    df['TotalCharges'] = df['TotalCharges'].fillna(df['TotalCharges'].median())
    
    # Feature Engineering
    # 1. Create tenure-related features
    df['tenure_group'] = pd.qcut(df['tenure'], 4, labels=['0-1 year', '1-3 years', '3-5 years', '5+ years'])
    
    # 2. Create average monthly charge
    df['avg_monthly_charges'] = df['TotalCharges'] / (df['tenure'] + 0.1)
    
    # 3. Create charge difference
    df['charge_diff'] = df['MonthlyCharges'] - df['avg_monthly_charges']
    
    # 4. Create service count feature
    service_columns = ['PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 
                       'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies']
    
    df['service_count'] = df.apply(count_services, axis=1)
    
    # 5. Create binary flags for premium services
    df['has_premium_tech'] = df['TechSupport'].apply(lambda x: 1 if x == 'Yes' else 0)
    df['has_premium_security'] = df['OnlineSecurity'].apply(lambda x: 1 if x == 'Yes' else 0)
    df['has_premium_support'] = df[['OnlineBackup', 'DeviceProtection']].apply(
        lambda x: 1 if 'Yes' in x.values else 0, axis=1
    )
    df['has_streaming'] = df[['StreamingTV', 'StreamingMovies']].apply(
        lambda x: 1 if 'Yes' in x.values else 0, axis=1
    )
    
    # 6. Create interaction features
    df['tenure_x_monthly'] = df['tenure'] * df['MonthlyCharges']
    
    # Binary encoding for Yes/No columns
    binary_cols = ['Partner', 'Dependents', 'PhoneService', 'PaperlessBilling']
    for col in binary_cols:
        df[col] = df[col].map({'Yes': 1, 'No': 0})
    
    # One-hot encoding for categorical columns
    cat_cols = ['gender', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup',
                'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies',
                'Contract', 'PaymentMethod', 'tenure_group']
    
    df = pd.get_dummies(df, columns=cat_cols, drop_first=True)
    
    # Ensure all required columns are present
    for col in feature_columns:
        if col not in df.columns:
            df[col] = 0
    
    # Select only the required columns in the correct order
    df = df[feature_columns]
    
    # Scale the features
    df_scaled = scaler.transform(df)
    
    return df_scaled

# Save the preprocessing function
joblib.dump(preprocess_for_prediction, 'improved_churn_preprocess.pkl')

print("Preprocessing function saved successfully!")

## 7. Model Comparison Summary

In [ ]:
# Create a summary of model performance
model_summary = pd.DataFrame({
    'Model': list(results.keys()),
    'Accuracy': [results[model]['accuracy'] for model in results],
    'Precision (Churn)': [results[model]['precision_churn'] for model in results],
    'Recall (Churn)': [results[model]['recall_churn'] for model in results],
    'F1 Score (Churn)': [results[model]['f1_churn'] for model in results]
}).sort_values('F1 Score (Churn)', ascending=False)

print("Model Performance Summary:")
display(model_summary)

# Plot model comparison
plt.figure(figsize=(12, 8))
model_summary.set_index('Model').plot(kind='bar', figsize=(12, 8))
plt.title('Model Performance Comparison')
plt.ylabel('Score')
plt.xticks(rotation=45)
plt.legend(loc='lower right')
plt.tight_layout()
plt.show()

# Compare with original model
print("\nOriginal Model Performance:")
with open('classification_report.txt', 'r') as f:
    original_report = f.read()
print(original_report)

print("\nBest New Model Performance:")
print(classification_report(y_test, y_pred))

print("\nImprovement Summary:")
print("1. Added feature engineering to create more predictive variables")
print("2. Addressed class imbalance using SMOTE")
print("3. Compared multiple algorithms to find the best performer")
print("4. Performed hyperparameter tuning to optimize model performance")
print("5. Created a comprehensive preprocessing pipeline for new data")